In [1]:
import pandas as pd

def get_invoices():
    pd.set_option('display.max_rows', 500)
    return pd.read_parquet('./invoices_facturx.parquet')

invoices = get_invoices()

In [2]:
#import numpy.ndarray
import numpy as np
  
def printall(m,mkey='',space='->'):
    for key, value in m.items():
        #print (type(value))
        if not isinstance(value, type(None)):
            #print(space + mkey, ' {0}: {1}'.format(key, value))
            if isinstance(value, dict):
                #print(space + (str(type(value))) + str(value.keys()))
                printall(value, mkey + "/" + key, space + '->')
            else:
                if isinstance(value,np.ndarray):
                    print('ARRAY',len(value), value)
                print(space + mkey + '/{0}: {1}'.format(key, value) + '(' + str(type(value)) + ')') 
 
def printpaths(m,mkey='',space='->'):
    for key, value in m.items():
        #print (type(value))
        if not isinstance(value, type(None)):
            #print(space + mkey, ' {0}: {1}'.format(key, value))
            if isinstance(value, dict):
                print(key,value)
                #print(key + space + (str(type(value))) + str(value.keys()))
                #printall(value, mkey + "/" + key, space + '->')


def printNotNone(val):
    if isinstance(val, dict):
        for key, value in val.items():
            if not isinstance(value, type(None)):
                print('{0}: {1}'.format(key, value)) 
    else:
        print(val)

def getDictNotNone(val):
    #print('getDictNotNone',type(val))
    if isinstance(val, dict):
        r = dict()
        #print('getDictNotNone',r.keys())
        for key, value in val.items():
            #print('getDictNotNone',key,value)
            if not isinstance(value, type(None)):
                r[key]=value
        return r       
    else:
        return dict        

def getPathValue(m,path=''):
    if(len(path)<2):
        return m
    splits = path[1:].split("/", 1)
    value=m.get(splits[0])
    #print(splits,value)
    if not isinstance(value, type(None)):
        if len(splits) > 1:
            return getPathValue(value,path='/' + splits[1])
    return value            
  
def getSimpleElementValue(d,key):
    __name__='getSimpleElementValue'
    #print(__name__,key,d)
    value=d[key]
    #print(value)
    #print(__name__,key,value, type(value))
    return key,value   
    
def getElementValue(d,key):
    #print(d)
    value=d[key]
    #print(value)
    #print(key,value, type(value))
    if isinstance(value, dict):
        if 'value' in value:
            return key,value['value']
        else: 
            #print("getElementValue",key)
            #print("getElementValue",value.keys())
            #print("getElementValue",value)
            #return key,getElementValue(value,list(value.keys())[0])
            return key,value
    elif isinstance(value,np.ndarray):
        #print(len(value),type(value[0]))
        if 'value' in value[0]:
            return value[0]['value']
        else:
            #print(value[0])
            return getElementValue(value[0],list(value[0].keys())[0])
    elif not isinstance(value, type(None)):
        return key,str(value)
    else:
        return None
            
    
def getElementValue1(d,key):
    #print(d)
    value=d[key]
    #print(value)
    #print(key,value, type(value))
    if isinstance(value, dict):
        if 'value' in value:
            return value['value']
        else: 
            print(key)
            print(value.keys())
            print(value)
            return getElementValue(value,list(value.keys())[0])
    elif isinstance(value,np.ndarray):
        #print(len(value),type(value[0]))
        if 'value' in value[0]:
            return value[0]['value']
        else:
            #print(value[0])
            return getElementValue(value[0],list(value[0].keys())[0])
    elif not isinstance(value, type(None)):
        return str(value)
    else:
        return None
        
def freeze(d):
    if isinstance(d, dict):
        return frozenset((key, freeze(value)) for key, value in d.items())
    elif isinstance(d, list):
        return tuple(freeze(value) for value in d)
    return d            

In [3]:


def doit(m,path,hideUnused=False):
    __name__='doit'
    print(__name__,type(m))
    r = getPathValue(m, path)
    if isinstance(r, type(None)):
        return
    if hideUnused == True:
        r = getDictNotNone(r)
    print(r)    
    for key in r.keys():
        value=getElementValue(r,key)
        #print (key,value)
        text = widgets.Text(description = key,value = value, style=style, layout=layout)
        display(text)



def doit1(m,path,hideUnused=False):
    __name__='doit1'
    r = getPathValue(m, path)
    #print(__name__,type(r),r,hideUnused)
    if hideUnused == True:
        r = getDictNotNone(r)
    proc1(r,hideUnused)
       

    

## replace by doitElement ???????    
def addChildsElement(r,hideUnused=False): 
    __name__='addChildsElement'
    #print(__name__,r,type(r),hideUnused)
    if isinstance(r, dict):
        doitElement(r,hideUnused=hideUnused)
        for key, value in r.items():
            #print (type(value))
            addHeader(str(key))    
            if not isinstance(value, type(None)):
                #print(space + mkey, ' {0}: {1}'.format(key, value))
                if isinstance(value, dict):
                    #print("addChildsElement",hideUnused,value)
                    doitElement(r,hideUnused=hideUnused)
    else:
        print(__name__,type(r),'not implemented')

# facturx_widgets

In [27]:
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import Javascript, display
import uuid

style = {'description_width': '250px'}
layout = {'width': '500px'}

from collections import namedtuple
Field = namedtuple("Field", "widget path")


class FieldContainer1():
    def __init__(self):
        self.fields = []
    def add(self,widget=None,path=''):
        self.fields.append(Field(widget=widget, path=path))
    def get_by_name(self,name):
        result = []
        for field in fields:
            if field.widget.description == name:
                return field

class FieldContainer():
    def __init__(self):
        self.fields = {}
    def add(self,widget=None,path=''):
        self.fields[widget.description] = (Field(widget=widget, path=path))
    def get_by_name(self,name):
        return self.fields[name]      
            
            
class facturx_widgets():
    #layout = widgets.Layout(width='auto', height='40px') #set width and height
    #style = {'description_width': '150px'}
    #layout = {'width': '400px'}
    
    def __init__(self,m,path,hideUnused=False): 
        self.container = FieldContainer()
        self.addChilds(m,path,hideUnused=hideUnused)
        
    
    def addHeader(self,text='',bold=True, path='' ):
        #text = text + '   /   ' + path
        if bold == True:
            html = widgets.HTML(
            value="<b>" + text +  "</b>",
            description=' ',
            style=style, layout=layout
            )
        else:
            html = widgets.HTML(
            value=text,
            description=' ',
            style=style, layout=layout
            )
        display(html)  
        return html

    def addVisText(self,key='',value=''):
        text = widgets.Text(description = key,value = value, style=style, layout=layout)
        display(text)   
        return text
    
    def addVisCheckbox(self,key='',value=False):
        text = widgets.Checkbox(description = key,value = value, style=style, layout=layout)
        display(text)   
        return text
        
    def procArray(self,r,hideUnused=False,rkey='',path=''):   
        __name__='procArray'
        #print (__name__,type(r),rkey)
        #path = path + '/' + rkey
        #print(__name__,path)
        
        if not isinstance(r, np.ndarray):
            print(__name__,type(r),'not implemented')
            return
        for element in r:
            #print (__name__,type(element),element)
            self.doitElement(element,rkey=rkey,hideUnused=hideUnused,path=path)

    def procList(self,r,hideUnused=False,rkey='',path=''):   
        __name__='procList' 
        #print (__name__,type(r),rkey)
        for element in r:
            #print (__name__,type(element),element)
            self.doitElement(element,rkey=rkey,hideUnused=hideUnused)
 
    def proc1(self,r,hideUnused=False,rkey='',path=''):
        __name__='proc1'
        if not isinstance(r, dict):
            print(__name__,type(r),'not implemented')
            return
        #print(__name__,'rkey=' + rkey,r.keys())
    
        if len(rkey) > 0: 
            path = path + '/' + rkey
        #print(__name__,path)
    
        #if(path == '/supplyChainTradeTransaction//includedSupplyChainTradeLineItem/associatedDocumentLineDocument'):
        #    print('FOUND',path)
    
        if hideUnused == True:
            r = getDictNotNone(r)
        
        self.proc1_childs(r,hideUnused,rkey,path)
        
    def proc_tmp(self):        
        if(path == '/supplyChainTradeTransaction//includedSupplyChainTradeLineItem/associatedDocumentLineDocument'):  
            outs2 = widgets.Output()
            with outs2:
                clear_output()   
                display(outs2)
                self.proc1_childs(r,hideUnused,rkey,path)
        else:
            self.proc1_childs(r,hideUnused,rkey,path)
            
    def proc1_childs(self,r,hideUnused=False,rkey='',path=''):   
        __name__='proc1_childs'
        ## option: remove value !!!
        if not 'value' in r:
            self.addHeader(rkey,bold=False, path=path)
            #print(__name__,path)
        for key in r.keys():
            #value=getElementValue(r,key)
            kk,value=getSimpleElementValue(r,key)
            #print(__name__,key,kk,type(value))
            
            if not isinstance(value, type(None)):
                if isinstance(value, dict):
                    self.proc1(value,hideUnused=hideUnused,rkey=kk,path=path)
                else:
                    if key == 'value' and len(rkey) > 0:
                        key=rkey
                    self.doitElement(value,rkey=key,hideUnused=hideUnused,upkey=kk,path=path)
            else:
                
                self.addVisText(key=key,value=value)  
 
        
    
    def doitElement(self,r,rkey='',hideUnused=False,upkey='',path=''):
        __name__='doitElement'
        #print(__name__,hideUnused,'type:',type(r),'rkey:',rkey,'upkey',upkey)
        #if isinstance(r, type(None)): 
        #    print("doitElement",type(r),hideUnused)
        #    return 
        #path = path + '/' + rkey
        #print(__name__,path)
        
        if isinstance(r, tuple):
            if isinstance(r[1], str):
                #addHeader(rkey)
                self.addVisText(key=r[0],value=r[1])
            else:
                #rkey=rkey + '.'+ r[0]
                #print(__name__,rkey)
                self.addHeader(rkey + __name__,bold=False)
                self.doitElement(r[1],rkey=rkey,hideUnused=hideUnused)
        elif isinstance(r, str):    
            if(rkey == 'value'):
                #print(__name__,rkey,upkey)
                widget=self.addVisText(key=upkey,value=r)
            else:
                widget=self.addVisText(key=rkey,value=r)  
            self.container.add(widget=widget,path=path)    
        elif isinstance(r, type(None)): 
            if hideUnused==False: 
                self.addVisText(key=rkey,value=r)
            return
        elif isinstance(r, dict):
            self.proc1(r,rkey=rkey,hideUnused=hideUnused,path=path)    
        elif isinstance(r,np.ndarray):    
            self.procArray(r,hideUnused=hideUnused,rkey=rkey,path=path) 
        elif isinstance(r, bool):  
            widget=self.addVisCheckbox(key=rkey,value=r)
            self.container.add(widget=widget,path=path)
        elif isinstance(r, int) or isinstance(r, float) :  
            #print(path)
            widget=self.addVisText(key=rkey,value=str(r))  
            self.container.add(widget=widget,path=path)
        else:
            print(__name__,type(r),'not implemented')

    def addChilds(self,m,path,hideUnused=False):
        __name__='addChilds'
        #print(__name__,hideUnused)
        #print(__name__,hideUnused,type(m),m)
        r = getPathValue(m, path)
        #print(__name__,hideUnused,type(r),r)
        self.doitElement(r,hideUnused=hideUnused,path=path)
        return    

In [5]:
from collections import namedtuple
MyStruct = namedtuple("MyStruct", "widget path")
m = MyStruct(widget="foo", path="bar")

In [6]:
#m.path

In [42]:
#len(getPathValue(invoices.iloc[int(invoice.value)],'/supplyChainTradeTransaction/includedSupplyChainTradeLineItem'))

#getPathValue(invoices.iloc[14],'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty/definedTradeContact/')[0]['personName']['value']
#getPathValue(invoices.iloc[14],'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty/definedTradeContact/')[0]['personName']['value']

'Stéphanie Hoarau'

In [8]:
#getPathValue(invoices.iloc[int(invoice.value)],'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty')[0].keys()
#create_field_container(invoices.iloc[int(invoice.value)],'/supplyChainTradeTransaction/includedSupplyChainTradeLineItem', hideUnused=True) 
#getPathValue(invoices.iloc[int(invoice.value)],'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty')

In [9]:
#invoices.iloc[15],pathtext.value, mytestfilter.value

In [28]:
outs = widgets.Output()
def create_field_container(x,path,hideUnused=True):
    with outs:
        clear_output()
        widget = facturx_widgets(x,path,hideUnused) 
        return widget
        
def mytest(x):    
    global _widget
    _widget = create_field_container(invoices.iloc[int(invoice.value)],pathtext.value, mytestfilter.value)  
        
#display(outs)

mytestfilter = widgets.Dropdown(options={True, False},value=True, description="FILTER", style=style, layout=layout)
invoice = widgets.Text(description = 'invoice',value = '15', style=style, layout=layout)
pathtext = widgets.Dropdown(options={'/exchangedDocumentContext', '/exchangedDocument', '/supplyChainTradeTransaction', 
                                     '/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty' , 
                                     '/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty',
                                    '/supplyChainTradeTransaction/applicableHeaderTradeAgreement'}, value = '/exchangedDocument',description="FILTER", style=style, layout=layout)
pathtext.observe(mytest, names='value')
invoice.on_submit(mytest)
display(invoice,mytestfilter,pathtext,outs)
create_field_container(invoices.iloc[int(invoice.value)],'/exchangedDocument', hideUnused=True)  

Text(value='15', description='invoice', layout=Layout(width='500px'), style=DescriptionStyle(description_width…

Dropdown(description='FILTER', index=1, layout=Layout(width='500px'), options=(False, True), style=Description…

Dropdown(description='FILTER', index=4, layout=Layout(width='500px'), options=('/exchangedDocumentContext', '/…

Output()

In [11]:
#_widget.fields.fields

In [12]:
import hashlib
import json

#from collections import OrderedDict

with open('asset.json') as json_file:
    asset = json.load(json_file)
    #asset=OrderedDict(asset)
    
facturx_widgets(asset,'/',hideUnused=False)


def get_value(op):
    try:
        return op
    except Exception as e:
        return ''

#_invoice = invoices.iloc[15]

def assign_invoice_to_asset(_invoice,asset):
    #print(asset)

    asset['hash'] = hash(repr(sorted(_invoice.items()))) 
    monetary = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeSettlement/specifiedTradeSettlementHeaderMonetarySummation/')
    asset['invoiceNumber'] = getPathValue(_invoice,'/exchangedDocument/id')['value']
    asset['netto'] = monetary['taxBasisTotalAmount'][0]['value']
    asset['tax'] = monetary['taxTotalAmount'][0]['value']
    
    try:
        asset['taxExemptionReason']  = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeSettlement/applicableTradeTax')[0]['exemptionReason']['value']
    except Exception as e:
        asset['taxExemptionReason']  = ''
 
    asset['countryBuyer'] = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty/postalTradeAddress/countryID')['value']
    asset['countryOrigin']   = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty/postalTradeAddress/countryID')['value']
    
    #print(asset)    

#monetary['totalPrepaidAmount'][0]['value']


HTML(value='', description=' ', layout=Layout(width='500px'), style=DescriptionStyle(description_width='250px'…

Text(value='asset10', description='ID', layout=Layout(width='500px'), style=DescriptionStyle(description_width…

Text(value='x509::CN=User1@aabby-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=…

Text(value='x509::CN=User1@aabby-1.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=…

Text(value='0', description='hash', layout=Layout(width='500px'), style=DescriptionStyle(description_width='25…

...

Checkbox(value=False, description='sold', layout=Layout(width='500px'), style=DescriptionStyle(description_wid…

Checkbox(value=False, description='claimPaid', layout=Layout(width='500px'), style=DescriptionStyle(descriptio…

Text(value='', description='claimPaidBy', layout=Layout(width='500px'), style=DescriptionStyle(description_wid…

Text(value='', description='taxExemptionReason', layout=Layout(width='500px'), style=DescriptionStyle(descript…

Checkbox(value=False, description='taxReceived', layout=Layout(width='500px'), style=DescriptionStyle(descript…

In [13]:
outs = widgets.Output()
def create_field_container(x,path,hideUnused=True):
    with outs:
        clear_output()
        widget = facturx_widgets(asset,'/',hideUnused=False)
    return widget    
        
def mytest(x): 
    global _widget
    assign_invoice_to_asset(invoices.iloc[int(invoice.value)],asset)
    _widget = create_field_container(asset,'/')  
        
#display(outs)

invoice = widgets.Text(description = 'invoice',value = '15', style=style, layout=layout)
invoice.on_submit(mytest)
display(invoice,outs)

Text(value='15', description='invoice', layout=Layout(width='500px'), style=DescriptionStyle(description_width…

Output()

In [15]:
#_widget.container.fields['sold'].widget.value = False

In [17]:
#_widget.container.fields['sold'].widget

In [18]:
def changed(b):
    print(b)

#_widget.container.fields['sold'].widget.observe(changed)

NameError: name '_widget' is not defined

In [ ]:
#asset['countryReceiver']='DE'
#facturx_widgets(asset,'/',hideUnused=False)
#_widget.container.fields
#asset.keys()

In [19]:
#getPathValue(invoices.iloc[int(invoice.value)],'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty')

In [20]:
class SimpleInvoiceContainer():
    def __init__(self,path='/', asset=None):
        self.path = path
        self.asset = asset
        self.outs = widgets.Output()
        self.invoice = widgets.Text(description = 'invoice',value = '15', style=style, layout=layout)
        self.invoice.on_submit(self.mytest)
        display(self.invoice,self.outs)

    def create_field_container(self,x,path,hideUnused=True):
        with self.outs:
            clear_output()
            widget = facturx_widgets(self.asset,self.path,hideUnused=False)
        return widget    

    def mytest(self,x): 
        assign_invoice_to_asset(invoices.iloc[int(invoice.value)],self.asset)
        self._widget = self.create_field_container(self.asset,self.path)  
        
SimpleInvoiceContainer(path='/', asset=asset)

Text(value='15', description='invoice', layout=Layout(width='500px'), style=DescriptionStyle(description_width…

Output()

In [25]:
class SimpleContainer():
    def __init__(self,path='/', asset=None):
        self.path = path
        self.asset = asset
        self.outs = widgets.Output()
        self.invoice = widgets.Text(description = 'invoice',value = '14', style=style, layout=layout)
        self.invoice.on_submit(self.mytest)
        display(self.invoice,self.outs)

    def create_field_container(self,x,path,hideUnused=True):
        with self.outs:
            clear_output()
            widget = facturx_widgets(self.asset,path,hideUnused=hideUnused)
        return widget    

    def mytest(self,x): 
        #assign_invoice_to_asset(invoices.iloc[int(invoice.value)],self.asset)
        #getPathValue(invoices.iloc[int(invoice.value)],'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty')
        self.asset=invoices.iloc[int(self.invoice.value)]
        self._widget = self.create_field_container(self.asset,self.path)  
        
#SimpleContainer(path='/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty', asset=invoices.iloc[14])
SimpleContainer(path='/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty', asset=invoices.iloc[14])

Text(value='14', description='invoice', layout=Layout(width='500px'), style=DescriptionStyle(description_width…

Output()

In [22]:
#getPathValue(invoices.iloc[14],'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty')

{'id': None,
 'globalID': None,
 'name': {'value': 'Au bon moulin'},
 'description': None,
 'specifiedLegalOrganization': {'id': {'value': '99999999800010',
   'schemeID': '0002'},
  'tradingBusinessName': None},
 'definedTradeContact': array([{'personName': {'value': 'Tony Dubois'}, 'departmentName': None, 'telephoneUniversalCommunication': {'uriid': None, 'completeNumber': {'value': '+33\xa04\xa072\xa007\xa008\xa056'}}, 'emailURIUniversalCommunication': {'uriid': {'value': 'tony.dubois@aubonmoulin.fr', 'schemeID': 'SMTP'}, 'completeNumber': None}}],
       dtype=object),
 'postalTradeAddress': {'postcodeCode': {'value': '84340',
   'listID': None,
   'listVersionID': None},
  'lineOne': {'value': "1242 chemin de l'olive"},
  'lineTwo': None,
  'lineThree': None,
  'cityName': {'value': 'Malaucène'},
  'countryID': {'value': 'FR'},
  'countrySubDivisionName': None},
 'uriUniversalCommunication': None,
 'specifiedTaxRegistration': array([{'id': {'value': 'FR11999999998', 'schemeID': 'V